HSE, Applied Time Series Forecasitng , Winter 2023-24

<font color="green"> Lesson #13: Bonus Topics </font>

<span style="color:black; font-size: 12pt"></span>

Alexey Romanenko,
<font color="blue">alexromsput@gmail.com</font>

# Business Value from Forecasting
 Business case: your company launched new Demand Forecasting System. What couldn't be as a business-value due to this implementation?
  
  a) free money due less overforecasting or less underforecasting
  
  b) free resources of analytics (less cost of support resources)
  
  c) increase transparancy and consistency in decision making processes (new money will come)
  
  d) speed up decision making process (more money due to fast reaction, more frequent decision making process)

#Demand restoration
 - <a href ='https://github.com/aromanenko/ATSF/tree/main/e2e%20demand%20forecasting%20pipeline' > theoretical part </a>

 -  <a href = 'https://github.com/aromanenko/ATSF/blob/main/e2e%20demand%20forecasting%20pipeline/src/demand_restoration.py'> example of production code </a>



<!-- # Implementation of Demand Forecastin Systems
 * True but expensive: AB test (regarding decision-making process)
 * Acceptance criteria: Designer of DFS Acceptance Calculator.xlsx
 * Go-live: RealCase_accuracy_report_june_2019_v2.xlsx  -->

HW3 Solution
  - <a href = 'https://github.com/aromanenko/ATSF/blob/wip/hw3_solution_example.ipynb'> example </a>

# Using of python packages for time series forecasting

https://github.com/aromanenko/ATSF/wiki/List-of-packages-for-Time-Series-Forecasting-in-Python